In [2]:
import numpy as np
import pandas as pd

##读取数据
data=pd.read_csv('data.csv',encoding='gb18030',index_col=0)
indicator=data.columns.tolist()   ##指标个数
project=data.index.tolist()    ##方案数、评价主体
value=data.values
print(indicator)
print(project)
print(value)
data.head()

['油耗', '功率', '费用', '安全性', '维护性', '操作性']
['本田', '奥迪', '桑塔纳', '别克']
[[ 5.   1.4  6.   3.   5.   7. ]
 [ 9.   2.  30.   7.   5.   9. ]
 [ 8.   1.8 11.   5.   7.   5. ]
 [12.   2.5 18.   7.   5.   5. ]]


,油耗,功率,费用,安全性,维护性,操作性
车型,,,,,,
本田,5,1.4,6,3,5,7
奥迪,9,2.0,30,7,5,9
桑塔纳,8,1.8,11,5,7,5
别克,12,2.5,18,7,5,5


In [3]:
###定义数据标准化函数。为了避免求熵值时对数无意义，对数据进行平移，对标准化后的数据统一加了常数0.001
def std_data(value,flag):
    for i in range(len(indicator)):
        #print(flag[i])
        if flag[i]=='+':
            value[:,i]=(value[:,i]-np.min(value[:,i],axis=0))/(np.max(value[:,i],axis=0)-np.min(value[:,i],axis=0))+0.001
        elif flag[i]=='-':
            value[:,i]=(np.max(value[:,i],axis=0)-value[:,i])/(np.max(value[:,i],axis=0)-np.min(value[:,i],axis=0))+0.001
    return value

In [4]:
#定义熵值法函数、熵值法计算变量的权重
def cal_weight(indicator,project,value):
    p= np.array([[0.0 for i in range(len(indicator))] for i in range(len(project))]) # 构造了一个全零矩阵，len(indicator)那么多列，len(project)那么多列                    
    ##print(p)
    for i in range(len(indicator)):
        p[:,i]=value[:,i]/np.sum(value[:,i],axis=0)
        
    e=-1/np.log(len(project))*sum(p*np.log(p))      #计算熵值
    g=1-e     # 计算一致性程度
    w=g/sum(g)     #计算权重
    return w

In [5]:
##数据标准化
flag=["-","+","-","+","+","+"]  ##表示指标为正向指标还是反向指标
std_value=std_data(value,flag)
std_value.round(3)

array([[1.001e+00, 1.000e-03, 1.001e+00, 1.000e-03, 1.000e-03, 5.010e-01],
       [4.300e-01, 5.460e-01, 1.000e-03, 1.001e+00, 1.000e-03, 1.001e+00],
       [5.720e-01, 3.650e-01, 7.930e-01, 5.010e-01, 1.001e+00, 1.000e-03],
       [1.000e-03, 1.001e+00, 5.010e-01, 1.001e+00, 1.000e-03, 1.000e-03]])

In [6]:
##结果
w=cal_weight(indicator,project,std_value)
w=pd.DataFrame(w,index=data.columns,columns=['权重'])
print("#######权重:#######")
print(w)
score=np.dot(std_value,w).round(2)
#print(score)
score=pd.DataFrame(score,index=data.index,columns=['综合得分']).sort_values(by =['综合得分'],ascending = False)
score

#######权重:#######
           权重
油耗   0.100233
功率   0.106534
费用   0.092712
安全性  0.094593
维护性  0.392790
操作性  0.213138


,综合得分
车型,
桑塔纳,0.61
奥迪,0.41
本田,0.30
别克,0.25


In [7]:
# 使用TOPSIS方法求解
# 正理想解和负理想解
cplus  = value.max(axis=0)
cminus = value.min(axis=0)
print("正理想解：",cplus, "负理想解：", cminus)

正理想解： [1.001 1.001 1.001 1.001 1.001 1.001] 负理想解： [0.001 0.001 0.001 0.001 0.001 0.001]


In [8]:
# 到正理想解和负理想解的距离
d1 = np.linalg.norm(value-cplus, axis=1)
d2 = np.linalg.norm(value-cminus, axis=1)
print(d1, d2)

[1.80277564 1.5915848  1.37187278 1.80277564] [1.5        1.57518067 1.52823366 1.5       ]


In [9]:
f1 = d2/(d1+d2)
print("TOPSIS的评价秩值为：", f1)

TOPSIS的评价秩值为： [0.45416346 0.49740995 0.52695778 0.45416346]


In [10]:
score=pd.DataFrame(f1,index=data.index,columns=['TOPSIS得分']).sort_values(by =['TOPSIS得分'],ascending = False)
score

,TOPSIS得分
车型,
桑塔纳,0.526958
奥迪,0.497410
本田,0.454163
别克,0.454163


In [11]:
# 使用灰色关联度进行分析
t=cplus-value   #计算参考序列与每个序列的差
mmin=t.min()
mmax=t.max()
rho=0.5  #分辨系数
xs=(mmin+rho*mmax)/(t+rho*mmax)  #计算灰色关联系数
f2=xs.mean(axis=1)  #求每一行的均值
print("\n关联系数=", xs,'\n关联度=',f2)  #显示灰色关联系数和灰色关联度


关联系数= [[1.         0.33333333 1.         0.33333333 0.33333333 0.5       ]
 [0.46666667 0.52380952 0.33333333 1.         0.33333333 1.        ]
 [0.53846154 0.44       0.70588235 0.5        1.         0.33333333]
 [0.33333333 1.         0.5        1.         0.33333333 0.33333333]] 
关联度= [0.58333333 0.60952381 0.58627954 0.58333333]


In [12]:
# 使用灰色关联度进行打分
score=pd.DataFrame(f2,index=data.index,columns=['灰色关联度得分']).sort_values(by =['灰色关联度得分'],ascending = False)
score

,灰色关联度得分
车型,
奥迪,0.609524
桑塔纳,0.586280
本田,0.583333
别克,0.583333


In [14]:
from scipy.stats import rankdata

In [15]:
# 使用秩和比方法计算评价值
R=[rankdata(value[:,i]) for i in np.arange(6)]  #求每一列的秩
R=np.array(R).T   #构造秩矩阵
print("\n秩矩阵为：\n",R)
RSR = R.mean(axis=1)
print("RSR=", RSR)


秩矩阵为：
 [[4.  1.  4.  1.  2.  3. ]
 [2.  3.  1.  3.5 2.  4. ]
 [3.  2.  3.  2.  4.  1.5]
 [1.  4.  2.  3.5 2.  1.5]]
RSR= [2.5        2.58333333 2.58333333 2.33333333]


In [16]:
# 使用秩和比方法得到结果
score=pd.DataFrame(RSR,index=data.index,columns=['灰色关联度得分']).sort_values(by =['灰色关联度得分'],ascending = False)
score

,灰色关联度得分
车型,
奥迪,2.583333
桑塔纳,2.583333
本田,2.500000
别克,2.333333


## 指标标准化、对缺失值或者零的处理
https://blog.csdn.net/yang978897961/article/details/79164829/ https://wenku.baidu.com/view/c435130f393567ec102de2bd960590c69ec3d882.html https://blog.csdn.net/weixin_43425784/article/details/107047869?utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-3.control&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-3.control
为了避免求熵值时对数无意义，对数据进行平移